In [1]:
!nvidia-smi

Thu May 30 13:19:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:18:00.0 Off |                  Off |
| 30%   24C    P8               7W / 450W |     13MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
if os.getcwd()[-8:]=='examples':
    os.chdir('..')
    
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # Possible GPUS

In [2]:
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
import torchdefenses as td
import torchdefenses.trainer as tr
from torchdefenses.transforms.cutout import Cutout

import src
from src.utils import *

### Settings

In [3]:
#### FOR TRAINING
EPOCHS = 100
BATCH_SIZE = 64

#### FOR MODELING
MODEL_NAME = "WRN16-4_WS"
DATA = "CIFAR10"
MEAN = [0.4930, 0.4875, 0.4548]
STD = [0.2475, 0.2445, 0.2637]
N_CLASSES = 10
LR = 0.1
RHO = 0.1

#### PATH SAVING
PATH = "./saved/"
NAME = f"{DATA}_{MODEL_NAME}_EPOCH{EPOCHS}_BS{BATCH_SIZE}_"
SAVE_PATH = PATH + NAME
DATA_PATH = '../DPTrainer/data/cifar_cond_bothinsample_weight10.npz'

### Data loader

In [4]:
data = np.load(DATA_PATH)

images = data['samples'][-50000:]
images = [Image.fromarray(np.array(img)) for img in images]
labels = torch.Tensor(data['labels'][-50000:]).type('torch.LongTensor')
labels = torch.argmax(labels, dim=1)

In [7]:
train_transform  = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                    Cutout(),
                   ])

dataset = PtDataset(images, labels, transform=train_transform)
public_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, 
                           drop_last=False, num_workers=16, pin_memory=True)

In [8]:
data = td.Datasets(data_name=DATA,
                   train_transform = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                    Cutout(),
                   ]),
                   val_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                   ]),
                   test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                   ]))
_, test_loader = data.get_loader(batch_size=BATCH_SIZE, drop_last_train=False, num_workers=16)

Files already downloaded and verified
Files already downloaded and verified
Data Loaded!
Train Data Length : 50000
Test Data Length : 10000


### Model & Optimizer

In [9]:
#### Load model
model = src.utils.load_model(model_name=MODEL_NAME, n_classes=N_CLASSES).cuda() # Load model
model = ModuleValidator.fix(model)
set_groupnorm_num_groups(model, num_groups=16)

ModuleValidator.validate(model, strict=False)

pytorch_total_params = sum(p.numel() for p in model.parameters())

print("model params: {:.4f}M".format(pytorch_total_params/1000000))

rmodel = td.RobModel(model, n_classes=N_CLASSES, normalization_used={'mean':MEAN, 'std':STD})

WRN16-4_WS is loaded.
model params: 2.7489M


### Start Training

In [ ]:
trainer = tr.Standard(rmodel)
trainer.setup(optimizer=f"SGD(lr={LR}, momentum=0.9, weight_decay=0.0005)",
              scheduler="Cosine", scheduler_type="Epoch",
              minimizer=f"SAM(rho={RHO})",
              n_epochs=EPOCHS, n_iters=len(public_loader)
             )
trainer.record_rob(public_loader, test_loader, eps=None)
trainer.fit(train_loader=public_loader, n_epochs=EPOCHS,
            save_path=SAVE_PATH, save_best={"Clean(Val)":"HBO"},
            save_type=None, save_overwrite=True, record_type="Epoch")

[Standard]
Training Information.
-Epochs: 100
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    initial_lr: 0.1
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
-Scheduler: <torch.optim.lr_scheduler.CosineAnnealingLR object at 0x7f280c54ed30>
-Minmizer: <torchdefenses.optim.minimizer.SAM object at 0x7f271288f640>
-Save Path: ./saved/CIFAR10_WRN16-4_WS_EPOCH100_BS64_
-Save Type: None
-Record Type: Epoch
-Device: cuda:0
-----------------------------------------------------------------------
Epoch   CALoss   CALoss_1   Clean(Tr)   Clean(Val)   lr       s/it     
1       1.0645   1.2347     50.7160     44.4300      0.1000   0.0743   
-----------------------------------------------------------------------
2       1.0790   1.3778     71.1140     59.6600      0.1000   0.0784   
-----------------------------------------------------------------------
3       0.3187   0.4939     80.7020     66.8700

### Evaluation

In [16]:
rmodel.load_dict(PATH+NAME+'/last.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 100), ('Iter', 500), ('CALoss', 0.004133778624236584), ('CALoss_1', 0.033079203218221664), ('Clean(Tr)', 99.934), ('Clean(Val)', 80.31), ('lr', 1.2335990856710008e-05), ('s/it', 0.10744806)])


80.31

In [17]:
rmodel.load_dict(PATH+NAME+'/best.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 93), ('Iter', 500), ('CALoss', 0.0031729331240057945), ('CALoss_1', 0.056504376232624054), ('Clean(Tr)', 99.944), ('Clean(Val)', 80.46), ('lr', 0.0007854209717842264), ('s/it', 0.230176634)])


80.46